In [1]:
import math
import random
import numpy as np
from mayavi import mlab
mlab.init_notebook()

def get_plane(A, B, C):
    n = np.cross(B - A , C - A)
    d = -(n[0] * A[0] + n[1] * A[1] + n[2] * A[2])
    return (n, d)

def mirror_point(a, b, c, d, point):
    x1, y1, z1 = point
    k =(-a * x1-b * y1-c * z1-d)/float((a * a + b * b + c * c))
    x2 = a * k + x1
    y2 = b * k + y1
    z2 = c * k + z1
    x3 = 2 * x2-x1
    y3 = 2 * y2-y1
    z3 = 2 * z2-z1
    return x3, y3, z3

Notebook initialized with ipy backend.


In [2]:
class Tetra:
    
    def __init__(self, sA, sB, sC, sD):
        self.A = np.array(sA)
        self.B = np.array(sB)
        self.C = np.array(sC)
        self.D = np.array(sD)
        
    def center(self):
        return ((self.A[0] + self.B[0] + self.C[0] + self.D[0])/4, 
        (self.A[1] + self.B[1] + self.C[1] + self.D[1])/4, 
        (self.A[2] + self.B[2] + self.C[2] + self.D[2])/4)
    
    def get_plot_points(self):
        points = []
        center = self.center()
        for vertex in [self.A, self.B, self.C, self.D]:
            points.append(vertex + 0.4 * (center - vertex))
        return points
    
    def reflect(self, Vertex):
        
        if Vertex == "A":
            plane = get_plane(self.B, self.C, self.D)
            new_vertex = mirror_point(*plane[0], plane[1], self.A)
            vertices = (new_vertex, self.B, self.C, self.D)
        elif Vertex == "B":
            plane = get_plane(self.A, self.C, self.D)
            new_vertex = mirror_point(*plane[0], plane[1], self.B)
            vertices = (self.A, new_vertex, self.C, self.D)
        elif Vertex == "C":
            plane = get_plane(self.A, self.B, self.D)
            new_vertex = mirror_point(*plane[0], plane[1], self.C)
            vertices = (self.A, self.B, new_vertex, self.D)
        elif Vertex == "D":
            plane = get_plane(self.A, self.B, self.C)
            new_vertex = mirror_point(*plane[0], plane[1], self.D)
            vertices = (self.A, self.B, self.C, new_vertex)
            
        return Tetra(*vertices)
    
    def plot_tetrahedron(self):
        x, y, z = [], [], []
        for vertex in [self.A, self.B, self.C, self.D]:
            x.append(vertex[0])
            y.append(vertex[1])
            z.append(vertex[2])
            
           
        mlab.clf()
        print(x,y,z)
        return mlab.points3d(np.array(x),np.array(y),np.array(z) , colormap="cool")
            
    def reflect_seq(self, seq, center_points = [], tetra_points = []):
        center_points.append(self.center())
        tetra_points.append(self.get_plot_points())
        if len(seq) == 1:
            reflection = self.reflect(seq)
            return reflection, center_points + [reflection.center()], tetra_points + [reflection.get_plot_points()]
        else:
            return self.reflect(seq[0]).reflect_seq(seq[1:], center_points, tetra_points)
        
    def reflect_seq_diagram(self, seq):
        final, centers, tetra_points = self.reflect_seq(seq)
        mlab.clf()
        mlab.close(all=True)
        for i in range(len(centers)):
            this_color = (random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1))
            # plot line connecting centers
            if i != 0:
                start, end = centers[i - 1], centers[i]
                mlab.plot3d([start[0], end[0]], [start[1], end[1]], [start[2], end[2]], color=(0,0,0), tube_radius=.03)
                
            # plot centers of mass
            mlab.points3d(*centers[i] , color=this_color, scale_factor=.3)
            
            # plot all vertices in all tetrahedrons
            
            for ip in range(len(tetra_points[i])):
                mlab.points3d(*tetra_points[i][ip] , color=this_color, scale_factor=.1)
                    
                for ip in range(len(tetra_points[i])):
                    mlab.points3d(*tetra_points[i][ip] , color=this_color, scale_factor=.1)
                
                    ips_to_connect = range(len(tetra_points[i]))[ip:]
                    for ip_to_connect in ips_to_connect:
                        start, end = tetra_points[i][ip], tetra_points[i][ip_to_connect]
                        mlab.plot3d([start[0], end[0]], [start[1], end[1]], [start[2], end[2]], color=this_color, tube_radius=.01)
        return mlab.points3d(self.center() , colormap="cool", scale_factor=.01)
            
       

In [3]:
t1 = Tetra(np.array((math.sqrt(2), 0, -1)), np.array((0, math.sqrt(2), -1)), np.array((0,0,-1)) ,np.array((0,0,0)))
mlab.clf()
mlab.close(all=True)
t1.reflect_seq_diagram("CABDCABD")
#t1.reflect_seq_diagram("ABCD")

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [44]:
# Define first tetrahedron with edges
t1 = Tetra(np.array((math.sqrt(2), 0, -1)), np.array((0, math.sqrt(2), -1)), np.array((0,0,-1)) ,np.array((0,0,0)))

# get the center of mass of the tetrahedron
print(t1.center())

# get a new tetrahedron by reflection the first one through edge C
t2 = t1.reflect("C")

# get the center of the tetrahedron which is created by sequentially reflecting tetrahedron 1 by the Sequence AA
# (first reflect t1 by A, and then reflect the result by A, and get the center of the final result)
print(t1.reflect_seq("ABABABAB")[1])

(0.3535533905932738, 0.3535533905932738, -0.75)
(0.3535533905932738, 0.3535533905932738, -0.75)
(-0.3535533905932738, 0.3535533905932738, -0.75)
(-0.3535533905932738, -0.3535533905932738, -0.75)
(0.3535533905932738, -0.3535533905932738, -0.75)
(0.3535533905932738, 0.3535533905932738, -0.75)
(-0.3535533905932738, 0.3535533905932738, -0.75)
(-0.3535533905932738, -0.3535533905932738, -0.75)
(0.3535533905932738, -0.3535533905932738, -0.75)
[(0.3535533905932738, 0.3535533905932738, -0.75), (-0.3535533905932738, 0.3535533905932738, -0.75), (-0.3535533905932738, -0.3535533905932738, -0.75), (0.3535533905932738, -0.3535533905932738, -0.75), (0.3535533905932738, 0.3535533905932738, -0.75), (-0.3535533905932738, 0.3535533905932738, -0.75), (-0.3535533905932738, -0.3535533905932738, -0.75), (0.3535533905932738, -0.3535533905932738, -0.75)]


(0.3535533905932738, 0.3535533905932738, -0.75)
[(0.3535533905932738, 0.3535533905932738, -0.75), (-0.3535533905932738, 0.3535533905932738, -0.75), (-0.3535533905932738, -0.3535533905932738, -0.75), (0.3535533905932738, -0.3535533905932738, -0.75), (0.3535533905932738, 0.3535533905932738, -0.75), (-0.3535533905932738, 0.3535533905932738, -0.75), (-0.3535533905932738, -0.3535533905932738, -0.75), (0.3535533905932738, -0.3535533905932738, -0.75), (0.3535533905932738, 0.3535533905932738, -0.75)]


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\r\t\n\x0b\…

In [66]:
mlab.show()

In [ ]:
t1.